# Tutorial EcoJupyter Tool
This tutorial helps you kickstart the use of the [EcoJupyter](https://github.com/g-uva/EcoJupyter) and [JupyterK8sTool](https://github.com/g-uva/JupyterK8sMonitor) tools.

## Before we start
Before we start, you must run from the menu: `Run > Run All Cells`. Why? This Notebook contains some nice UI that is only accessible after the cells are executed.

## Step 1: install `EcoJupyter`
First, let's install the tool using the `pip` command! 👇

In [ ]:
!pip install --upgrade ecojupyter

If you see the message `Successfully installed ecojupyter-<version>`, congratulations! `EcoJupyter` has been successfully installed in this notebook. 👏

You can open your newly installed plugin by:
1. Refreshing the page.
2. Click on `View > Activate Command Palette`.
3. Look for `Open GreenDIGIT Dashboard`.

## Step 2: Install monitoring agent `JupyterK8sMonitor`
You must run this script in order to download and run the tool that will expose the metrics that will later be read by `EcoJupyter` tool.


In [ ]:
import ipywidgets as widgets
from IPython.display import display
import subprocess

output = widgets.Output()

def run_step(description, command, shell=False, env=None):
    print(f"---- {description} ----")
    try:
        result = subprocess.run(
            command,
            shell=shell,
            check=True,
            capture_output=True,
            text=True,
            env=env
        )
        if result.stdout:
            print(result.stdout)
        if result.stderr:
            print(result.stderr)
    except subprocess.CalledProcessError as e:
        print(f"ERROR during: {description}")
        print(e.output)
        print(e.stderr)
        raise

def on_scaphandre_click(b):
    with output:
        output.clear_output()  # Clear previous outputs
        run_step("Downloading script...", [
            "curl", "-O",
            "https://raw.githubusercontent.com/g-uva/jupyterhub-scaphandre-monitor/refs/heads/master/scaphandre-prometheus-ownpod/install-scaphandre-prometheus.sh"
        ])
        run_step("Setting executable permission...", ["chmod", "+x", "install-scaphandre-prometheus.sh"])

        print("Starting installation process...\n")

        run_step("Create bin directory", ["mkdir", "-p", "/home/jovyan/.bin"])
        run_step("Move installer to bin", ["mv", "install-scaphandre-prometheus.sh", "/home/jovyan/.bin/"])
        # run_step("Change to bin dir", ["bash", "-c", "cd /home/jovyan/.bin"])
        # print("---- Done with Scaphandre installation ✅ -----")
        spinner.value = "<span style='color:green;'>✔️ Completed successfully!</span>"

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import subprocess
import threading
import os
import requests

spinner = widgets.HTML("<i>Idle</i>")
output = widgets.Output()

def run_step(description, command, shell=False, env=None):
    spinner.value = f"<i class='fa fa-spinner fa-spin'></i> {description}"
    subprocess.run(
        command,
        shell=shell,
        check=True,
        capture_output=True,
        text=True,
        env=env
    )

def check_scaphandre_and_prometheus():
    with output:
        output.clear_output()
        print("Checking Scaphandre and Prometheus installations...")
        
        scaphandre_path = "/usr/local/bin/scaphandre"
        prometheus_dir = "/home/jovyan/.bin/prometheus-unzipped"
        prometheus_config = "/home/jovyan/.bin/prometheus.yml"
        
        scaphandre_ok = os.path.isfile(scaphandre_path)
        prometheus_ok = os.path.isdir(prometheus_dir) and os.path.isfile(prometheus_config)
        
        if scaphandre_ok and prometheus_ok:
            print("✅ Scaphandre and Prometheus are already installed.")
        else:
            if not scaphandre_ok:
                print("❌ Scaphandre binary not found at", scaphandre_path)
            if not prometheus_ok:
                print(f"❌ Prometheus folder or config missing:\n- Folder: {prometheus_dir}\n- Config: {prometheus_config}")

def check_prometheus_running():
    with output:
        print("\nChecking if Prometheus server is running on localhost:9090...")
        try:
            response = requests.get("http://localhost:9090")
            if response.status_code == 200:
                print("✅ Prometheus server is already running.")
            else:
                print(f"⚠️ Prometheus server responded with status code {response.status_code}.")
        except requests.ConnectionError:
            print("❌ Prometheus server is NOT running on localhost:9090.")

def run_installation():
    try:
        home = os.path.expanduser("~")
        env = os.environ.copy()
        env["HOME"] = home

        run_step("APT update", ["sudo", "apt-get", "update"])
        run_step("Install dependencies", ["sudo", "apt-get", "install", "-y", "pkg-config", "libssl-dev", "lsof"])

        run_step(
            "Install Rust and build Scaphandre",
            """
            bash -c '
            curl --proto "=https" --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
            source $HOME/.cargo/env
            rustup install 1.65.0
            rustup override set 1.65.0
            git clone https://github.com/hubblo-org/scaphandre.git
            cd scaphandre
            cargo build --release
            sudo mv ./target/release/scaphandre /usr/local/bin/
            cd ~ && rm -rf scaphandre
            '
            """,
            shell=True
        )

        run_step("Start Scaphandre", "nohup scaphandre prometheus --address=0.0.0.0 --port=8081 --containers > /home/jovyan/.bin/scaphandre.log 2>&1 &", shell=True)

        run_step("Remove old Prometheus", ["sudo", "rm", "-rf", "/home/jovyan/.bin/prometheus-unzipped"])
        run_step("Download Prometheus", ["wget", "https://github.com/prometheus/prometheus/releases/download/v2.52.0/prometheus-2.52.0.linux-amd64.tar.gz", "-P", "/home/jovyan/.bin/"])
        run_step("Unzip Prometheus", ["tar", "xzf", "/home/jovyan/.bin/prometheus-2.52.0.linux-amd64.tar.gz", "-C", "/home/jovyan/.bin/"])
        run_step("Move Prometheus", ["mv", "/home/jovyan/.bin/prometheus-2.52.0.linux-amd64", "/home/jovyan/.bin/prometheus-unzipped"])
        run_step("Cleanup Prometheus archive", ["rm", "-rf", "/home/jovyan/.bin/prometheus-2.52.0.linux-amd64.tar.gz"])

        prometheus_config = """
global:
  scrape_interval: 5s

scrape_configs:
  - job_name: 'scaphandre-local'
    static_configs:
      - targets: ['localhost:8081']
"""
        config_path = "/home/jovyan/.bin/prometheus.yml"
        with open(config_path, "w") as f:
            f.write(prometheus_config)

        spinner.value = "<span style='color:green;'>✔️ Completed successfully!</span>"

    except Exception as e:
        spinner.value = f"<span style='color:red;'>❌ Installation failed: {e}</span>"

def start_prometheus():
    try:
        run_step("Start Prometheus", "nohup /home/jovyan/.bin/prometheus-unzipped/prometheus --config.file=/home/jovyan/.bin/prometheus.yml --web.listen-address=0.0.0.0:9090 > /home/jovyan/.bin/prometheus.log 2>&1 &", shell=True)
        spinner.value = "<span style='color:green;'>✔️ Prometheus started successfully!</span>"
    except Exception as e:
        spinner.value = f"<span style='color:red;'>❌ Failed to start Prometheus: {e}</span>"

def on_install_click(b):
    threading.Thread(target=run_installation).start()

def on_prometheus_click(b):
    threading.Thread(target=start_prometheus).start()

def on_check_click(b):
    check_scaphandre_and_prometheus()
    check_prometheus_running()

# UI Buttons
button_scaphandre = widgets.Button(description="Install Scaphandre", layout=widgets.Layout(width='200px'))
install_button = widgets.Button(description="Install Prometheus", layout=widgets.Layout(width='200px'))
prometheus_button = widgets.Button(description="Start Prometheus", layout=widgets.Layout(width='200px'))
check_button = widgets.Button(description="Check Status", layout=widgets.Layout(width='200px'))

button_scaphandre.on_click(on_scaphandre_click)
install_button.on_click(on_install_click)
prometheus_button.on_click(on_prometheus_click)
check_button.on_click(on_check_click)

display(widgets.VBox([
    widgets.HBox([button_scaphandre, install_button, prometheus_button, check_button]),
    spinner, output
]))


## Step 3: Run a Workload
If you do **not** have the `EcoJupyter` plugin installed and opened on the side, please follow the last instructions of Step 1.

If you do, then you're good to go! Let's spin up some stress test work by running the below cell.

> Note: if you do not see charts with data on the plugin, consider reloading the window. Sometimes it might take some time to synchronise.

In [ ]:
import multiprocessing
import threading
import time
import ipywidgets as widgets
from IPython.display import display, clear_output

# ----------- Definitions --------------

def cpu_stress(cores, duration, output_box):
    def cpu_load():
        while not stop_event.is_set():
            pass  # Busy wait
    stop_event = threading.Event()
    processes = []
    try:
        output_box.append_stdout(f"Starting {cores} CPU stress processes...\n")
        for _ in range(cores):
            p = multiprocessing.Process(target=cpu_load)
            p.start()
            processes.append(p)
        output_box.append_stdout(f"Stressing CPU for {duration} seconds...\n")
        time.sleep(duration)
    finally:
        stop_event.set()
        for p in processes:
            p.terminate()
        output_box.append_stdout("Stopped CPU stress test.\n\n")

def ram_stress(mb, duration, output_box):
    big_data = []
    try:
        output_box.append_stdout(f"Allocating ~{mb} MB RAM...\n")
        for _ in range(mb):
            big_data.append(bytearray(1024 * 1024))  # Allocate 1MB
        output_box.append_stdout(f"RAM allocation done. Holding for {duration} seconds...\n")
        time.sleep(duration)
    except MemoryError:
        output_box.append_stdout("MemoryError: Could not allocate requested memory.\n")
    finally:
        del big_data
        output_box.append_stdout("Released memory.\n\n")

def combined_stress(cores, mb, duration, output_box):
    # Start both CPU and RAM stress in threads so both run concurrently
    cpu_thread = threading.Thread(target=cpu_stress, args=(cores, duration, output_box))
    ram_thread = threading.Thread(target=ram_stress, args=(mb, duration, output_box))
    output_box.append_stdout("Running combined CPU and RAM stress...\n")
    cpu_thread.start()
    ram_thread.start()
    cpu_thread.join()
    ram_thread.join()
    output_box.append_stdout("Combined stress test complete.\n\n")

# ----------- Widgets & UI -------------

warning = widgets.HTML("<b style='color: darkred;'>Warning:</b> Be mindful about increasing the pre-defined values—"
                      "setting too high may crash your environment or cause instability.")

cpu_count = multiprocessing.cpu_count()
cpu_input = widgets.BoundedIntText(value=min(2, cpu_count), min=1, max=cpu_count,
                                   description="CPU cores:")
ram_input = widgets.BoundedIntText(value=500, min=10, max=20000, step=10,
                                   description="RAM (MB):")
dur_input = widgets.BoundedIntText(value=30, min=5, max=600, step=5,
                                   description="Duration (s):")

cpu_button = widgets.Button(description="Run CPU Stress Test", button_style="warning")
ram_button = widgets.Button(description="Run RAM Stress Test", button_style="warning")
both_button = widgets.Button(description="Run BOTH", button_style="danger")

output_box = widgets.Output()

def on_cpu_clicked(b):
    with output_box:
        clear_output()
        cpu_stress(cpu_input.value, dur_input.value, output_box)

def on_ram_clicked(b):
    with output_box:
        clear_output()
        ram_stress(ram_input.value, dur_input.value, output_box)

def on_both_clicked(b):
    with output_box:
        clear_output()
        combined_stress(cpu_input.value, ram_input.value, dur_input.value, output_box)

cpu_button.on_click(on_cpu_clicked)
ram_button.on_click(on_ram_clicked)
both_button.on_click(on_both_clicked)

controls = widgets.HBox([cpu_input, ram_input, dur_input])
buttons = widgets.HBox([cpu_button, ram_button, both_button])

display(warning, controls, buttons, output_box)